In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dense
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from tensorflow import keras
from sklearn.model_selection import train_test_split
import tensorflow.keras.regularizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,LSTM
from tensorflow.keras.datasets import imdb
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import numpy as np
import matplotlib as matplot
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
import math
# from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
import pickle
from tensorflow.keras.layers import GaussianNoise
from sklearn.metrics import mutual_info_score






# set up hyperparameters
# max_features = 50000
np.random.seed(44)

maxlen = 500
batch_size = 128
VOCAB_size = 4000
INPUT_SIZE=500

dataset = DataSet(VOCAB_size, maxlen=INPUT_SIZE, train_portion=0.7)
with open('l2wd/dataset_4000_70.pkl', 'wb') as f:
    pickle.dump(dataset, f)
(x_train, y_train), (x_test, y_test),(_,_) = dataset.get_data()

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

#with open('x_train_embedded.pkl', 'rb') as f:
    #x_train_embedded = pickle.load(f)
#x_train=x_train_embedded[0:35000]
#x_test=x_train_embedded[35000:]


model_s = tf.keras.Sequential()


model_s.add(Embedding(VOCAB_size, output_dim=32, input_length=500))


model_s.add(LSTM(60))


model_s.add(Dense(1, activation='sigmoid',trainable=False))
model_s.load_weights('paper(2 and3)/paper2/weights.580.hdf5')

opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
model_s.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

#train step
def custom_train_step(model, inputs, targets, l2_strength):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = tf.keras.losses.binary_crossentropy(targets, predictions)

        # Calculate L2 regularization term for LSTM kernel and recurrent weights
        if l2_strength > 0.0:
            lstm_weights = model.layers[1].trainable_weights  # Access the LSTM layer's weights
            l2_reg = tf.add_n([tf.nn.l2_loss(w) for i,w in enumerate(lstm_weights) if i==1])
            loss += l2_strength * l2_reg

    # Compute gradients
    trainable_weights = model.trainable_variables
    gradients = tape.gradient(loss, trainable_weights)



    # non update kernel and bias
    #for i, weight in enumerate(trainable_weights):
        #if i==2:
            #gradients[i] = tf.zeros_like(weight)


    opt.apply_gradients(zip(gradients, trainable_weights))
    return loss



def compute_mutual_information(predictions, actual_labels):
    
    flat_predictions = predictions.flatten()
    flat_actual_labels = actual_labels.flatten()
    
    mi = mutual_info_score(flat_predictions, flat_actual_labels)
    return mi


def custom_train_loop(model, x_train, y_train, x_test, y_test, epochs, batch_size,l2_strength):
    num_batches = len(x_train) // batch_size
    lo=[]
    ac=[]
    mi=[]
    for epoch in range(epochs):
        
        #a=[20*i for i in range(1,14)]

            epoch_loss = 0.0
            for batch in range(num_batches):
                batch_inputs = x_train[batch * batch_size : (batch + 1) * batch_size]
                batch_targets = y_train[batch * batch_size : (batch + 1) * batch_size]

                batch_targets = batch_targets.reshape(-1, 1)

                loss= custom_train_step(model, batch_inputs, batch_targets,l2_strength)
                epoch_loss += tf.reduce_mean(loss)
                
                path1='l2wd'+'/'+'weight.'+str(epoch+batch/num_batches)+'.h5'
                model.save(path1)

            avg_loss = epoch_loss / num_batches
            print("Epoch {}: Loss: {:.4f}".format(epoch + 1, avg_loss.numpy()))
            

            # Reshape y_test to (batch_size, 1) to match the model output shape
            y_test_reshaped = y_test.reshape(-1, 1)
            val_loss,val_accuracy = model.evaluate(x_test, y_test_reshaped, verbose=0)

            print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))
            print("Validation Loss: {:.4f}".format(val_loss))
            
            

            lo.append(val_loss)
            ac.append(val_accuracy)
    print(lo,ac)
    

    path='l2wd'+'/'+'losacc.pkl'
    with open(path,'wb') as f:
        pickle.dump([lo,ac],f)

#train
l2_strength = 0.0
custom_train_loop(model_s, x_train, y_train, x_test, y_test, epochs=400, batch_size=batch_size,l2_strength=l2_strength)


x_train shape: (35000, 500)
x_test shape: (15000, 500)


2023-08-31 02:55:29.963571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30341 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:0f:00.0, compute capability: 7.0
2023-08-31 02:55:30.892814: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 02:55:30.894840: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/sp

Epoch 1: Loss: 0.0052


2023-08-31 02:56:42.838581: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 02:56:42.840655: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 02:56:42.842249: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Validation Accuracy: 86.07%
Validation Loss: 1.1404
Epoch 2: Loss: 0.0044
Validation Accuracy: 86.07%
Validation Loss: 1.1603
Epoch 3: Loss: 0.0042
Validation Accuracy: 86.09%
Validation Loss: 1.1859
Epoch 4: Loss: 0.0064
Validation Accuracy: 86.07%
Validation Loss: 1.0410
Epoch 5: Loss: 0.0045
Validation Accuracy: 85.97%
Validation Loss: 1.1283
Epoch 6: Loss: 0.0040
Validation Accuracy: 86.02%
Validation Loss: 1.1324
Epoch 7: Loss: 0.0037
Validation Accuracy: 85.93%
Validation Loss: 1.2037
Epoch 8: Loss: 0.0038
Validation Accuracy: 85.85%
Validation Loss: 1.2095
Epoch 9: Loss: 0.0062
Validation Accuracy: 86.01%
Validation Loss: 1.0841
Epoch 10: Loss: 0.0059
Validation Accuracy: 86.02%
Validation Loss: 1.0711
Epoch 11: Loss: 0.0036
Validation Accuracy: 85.94%
Validation Loss: 1.1763
Epoch 12: Loss: 0.0046
Validation Accuracy: 86.01%
Validation Loss: 1.1496
Epoch 13: Loss: 0.0038
Validation Accuracy: 85.97%
Validation Loss: 1.1788
Epoch 14: Loss: 0.0038
Validation Accuracy: 85.91%
Valid


KeyboardInterrupt



In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import math
import numpy as np
import math
import numpy
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, Model
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
import multiprocessing
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras import models, Model
import numpy as np

class DataSet():
    def __init__(self, num_words=None, seed=113, maxlen=100, train_portion=0.5, long_first=False):
        """
        :param num_words: Size of vocabulary, all words out-of-vocabulary will be marked as [unk]
        :param seed: Seed for randomly shuffling dataset
        :param maxlen: Maximum length of review, reviews longer than maxlen will be truncated, reviews shorter than
        maxlen will be padded to maxlen
        :param train_portion: Portion of dataset to allocate to training data, test data will be 1-train_portion
        :param long_first: Sort reviews by length
        """
        # retrieve IMDb data, x is a sequence containing movie review,
        # y is a label indicating if it is positive or negative sentiment
        (self.x_train, self.y_train), (self.x_test, self.y_test) = imdb.load_data(num_words=num_words, seed=seed)

        if long_first:
            self.y_train = [y for _, y in sorted(zip(self.x_train, self.y_train), key=lambda s: len(s[0]), reverse=True)]
            self.x_train = sorted(self.x_train, key=len, reverse=True)

            self.y_test = [y for _, y in sorted(zip(self.x_test, self.y_test), key=lambda s: len(s[0]), reverse=True)]
            self.x_test = sorted(self.x_test, key=len, reverse=True)

        self.total_length = [len(x) for x in self.x_train]
        temp = [len(x) for x in self.x_test]
        self.total_length.extend(temp)

        # padding sequences to all be of the same length
        self.x_train = pad_sequences(self.x_train, maxlen=maxlen, padding='post', truncating='post')
        self.x_test = pad_sequences(self.x_test, maxlen=maxlen, padding='post', truncating='post')

        self.split_data(train_portion)

        self.word_index = imdb.get_word_index()
        self.word_to_index = {word:id + 3 for word, id in self.word_index.items()}
        self.word_to_index["[pad]"] = 0
        self.word_to_index["[start]"] = 1
        self.word_to_index["[unk]"] = 2
        self.word_to_index["[unused]"] = 3
        self.index_to_word = {i:word for (word, i) in self.word_to_index.items()}
        self.form_vocab()

    def form_vocab(self):
        i2w_vocab = {}
        w2i_vocab = {}

        for sentence in self.x_train:
            for id in sentence:
                if id not in i2w_vocab:
                    word = self.index_to_word[id]
                    i2w_vocab[id] = word
                    w2i_vocab[word] = id

        for sentence in self.x_test:
            for id in sentence:
                if id not in i2w_vocab:
                    word = self.index_to_word[id]
                    i2w_vocab[id] = word
                    w2i_vocab[word] = id
        self.i2w_vocab = i2w_vocab
        self.w2i_vocab = w2i_vocab

    def get_data(self):
        return (self.x_train, self.y_train), (self.x_test, self.y_test), (self.train_length, self.test_length)

    def get_vocab_length(self):
        return len(self.i2w_vocab)

    # splits data into ratio train:test -> (train_portion:1-train_portion)
    def split_data(self, train_portion):
        x = np.concatenate((self.x_train, self.x_test), axis=0)
        y = np.concatenate((self.y_train, self.y_test), axis=0)
        self.train_length, self.test_length = self.total_length[:math.floor(train_portion * len(x))], \
                                              self.total_length[math.floor(train_portion * len(x)):]
        self.x_train, self.x_test = x[:math.floor(train_portion * len(x))], \
                                    x[math.floor(train_portion * len(x)):]
        self.y_train, self.y_test = y[:math.floor(train_portion * len(y))], \
                                    y[math.floor(train_portion * len(y)):]
        return None

2023-08-31 02:55:12.642546: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
with open('l2wd/losacc.pkl', 'rb') as f:
    x_train_embedded = pickle.load(f)

EOFError: Ran out of input

In [18]:
import os
scores = {} # scores is an empty dict already
if os.path.getsize(path) > 0:      
    with open(path, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()


In [27]:
dataset = DataSet(VOCAB_size, maxlen=INPUT_SIZE, train_portion=0.7)
with open('l2wd/dataset_4000_70.pkl', 'wb') as f:
    pickle.dump(dataset, f)
(x_train, y_train), (x_test, y_test),(_,_) = dataset.get_data()
loss=[]
acc=[]
for j in range(400):
    path='l2wd/weight.'+str(float(j+1))+'.'+'h'+'5'
    model=models.load_model(path)
    y_test_reshaped = y_test.reshape(-1, 1)
    val_loss,val_accuracy = model.evaluate(x_test, y_test_reshaped, verbose=0)
    loss.append(val_loss)
    acc.append(val_accuracy)

    print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))
    print("Validation Loss: {:.4f}".format(val_loss))

2023-08-31 00:54:35.291669: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 00:54:35.293781: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 00:54:35.295416: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Validation Accuracy: 86.09%
Validation Loss: 1.1398


2023-08-31 00:54:41.243364: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 00:54:41.245846: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 00:54:41.247890: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Validation Accuracy: 86.03%
Validation Loss: 1.1599


2023-08-31 00:54:47.304727: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 00:54:47.306781: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 00:54:47.308356: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

Validation Accuracy: 85.75%
Validation Loss: 1.8316


2023-08-31 01:35:37.307663: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 01:35:37.309609: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 01:35:37.311199: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Validation Accuracy: 85.73%
Validation Loss: 1.8423


2023-08-31 01:35:43.307464: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-31 01:35:43.309508: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-31 01:35:43.311134: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Validation Accuracy: 86.35%
Validation Loss: 1.6255


In [29]:
with open('l2wd/loss.pck','wb') as f:
    pickle.dump(loss,f)

In [30]:
with open('l2wd/accu.pck','wb') as f:
    pickle.dump(acc,f)